## play with dbs

#### imports

In [19]:
import sys
sys.path.append('library')
import cnfg
from connection import *
from survey import *
from __future__ import division

#### read server config file

In [32]:
db_config = cnfg.load(".fff_config")

#### start ssh tunnels and db connections

In [29]:
# connect to push db
push_ssh = define_ssh_tunnel(db_config,db_config['push_server_host'])
push_ssh.start()
push_con = open_db_connection(db_config,push_ssh)

# connect to prod db
prod_ssh = define_ssh_tunnel(db_config,db_config['prod_server_host'])
prod_ssh.start()
prod_con = open_db_connection(db_config,prod_ssh)

#### run queries for actives, skin tone surveys, spring select surveys

In [35]:
actives = return_actives(push_con)
skin_tone_dict = return_skin_tone_dict(prod_con,actives)
spring_select_dict = return_spring_select_dict(prod_con,actives)

#### parse skus

In [36]:
skus = return_skus(sku_key, actives, skin_tone_dict, spring_select_dict)

In [37]:
total_active = len(actives)
print 'Actives :',total_active
total_skus = 0
lights = 0
darks = 0
for sku in sku_order:
    print sku,':',skus[sku]
    total_skus += skus[sku]
    if sku[-1] == '1' or sku[-2] == '1':
        lights += skus[sku]
    if sku[-1] == '2' or sku[-2] == '2':
        darks += skus[sku]
print 'Actives without SKU:', len(actives) - total_skus
print 'Light kit :', lights, "(",round(((lights/total_skus)*100),2),"%)"
print 'Dark kit :', darks, "(",round(((darks/total_skus)*100),2),"%)"

Actives : 99234
Q-1601A-G1 : 71336
Q-1601A-G2 : 2052
Q-1601A-1A : 4195
Q-1601A-1B : 2585
Q-1601A-1C : 4642
Q-1601A-1D : 4316
Q-1601A-2A : 53
Q-1601A-2B : 36
Q-1601A-2C : 101
Q-1601A-2D : 76
Actives without SKU: 9842
Light kit : 87074 ( 97.41 %)
Dark kit : 2318 ( 2.59 %)


#### shut down ssh tunnels

In [8]:
push_ssh.stop()
prod_ssh.stop()